# Model Use: *LSTM*

# 1. Environement Setup

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

file_path = 'normalized_output.csv'

# 2. Data Processing

In [2]:
# Load the data
data = pd.read_csv(file_path)

# Function to convert string representation of a list to an actual list
def convert_string_to_list(string):
    return np.fromstring(string[1:-1], sep=',') 

X = np.array([convert_string_to_list(row) for row in data['ph_seq_encoded']])
y_note = np.array([convert_string_to_list(row) for row in data['note_seq_encoded']])
y_dur = np.array([convert_string_to_list(row) for row in data['ph_dur']])

# Reshape the arrays
X_reshaped = X.reshape((X.shape[0], X.shape[1], 1))
y_note_reshaped = y_note.reshape((y_note.shape[0], y_note.shape[1], 1))
y_dur_reshaped = y_dur.reshape((y_dur.shape[0], y_dur.shape[1], 1))

# print(X_reshaped[0].dtype, y_note_reshaped[0].dtype, y_dur_reshaped[0].dtype)

# Split the data into training and test sets
X_train, X_test, y_note_train, y_note_test, y_dur_train, y_dur_test = train_test_split(
    X_reshaped, y_note_reshaped, y_dur_reshaped, test_size=0.3, random_state=42)

y_dur_train.shape

# 3. Model Training

In [3]:
# Input layer
input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]))

# LSTM layers
lstm1 = LSTM(64, return_sequences=True)(input_layer)
lstm2 = LSTM(32, return_sequences=True)(lstm1)

# Output layers
y_note_output = Dense(10, activation='relu')(lstm2)
y_note_output = Dense(1, activation='sigmoid', name='y_note_output')(y_note_output)

y_dur_output = Dense(10, activation='relu')(lstm2)
y_dur_output = Dense(1, activation='sigmoid', name='y_dur_output')(y_dur_output)

# Building the model
model = Model(inputs=input_layer, outputs=[y_note_output, y_dur_output])

# Compiling the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Training the model
history = model.fit(X_train, [y_note_train, y_dur_train], epochs=200, batch_size=64, validation_split=0.2)

# Evaluate the model on the test set
test_loss = model.evaluate(X_test, [y_note_test, y_dur_test])



Epoch 1/200


36/36 [==============================] - 15s 270ms/step - loss: 0.0822 - y_note_output_loss: 0.0208 - y_dur_output_loss: 0.0615 - val_loss: 0.0656 - val_y_note_output_loss: 0.0146 - val_y_dur_output_loss: 0.0510
Epoch 2/200
36/36 [==============================] - 8s 236ms/step - loss: 0.0743 - y_note_output_loss: 0.0154 - y_dur_output_loss: 0.0589 - val_loss: 0.0646 - val_y_note_output_loss: 0.0144 - val_y_dur_output_loss: 0.0502
Epoch 3/200
36/36 [==============================] - 8s 224ms/step - loss: 0.0735 - y_note_output_loss: 0.0152 - y_dur_output_loss: 0.0583 - val_loss: 0.0637 - val_y_note_output_loss: 0.0142 - val_y_dur_output_loss: 0.0496
Epoch 4/200
36/36 [==============================] - 8s 217ms/step - loss: 0.0729 - y_note_output_loss: 0.0151 - y_dur_output_loss: 0.0577 - val_loss: 0.0631 - val_y_note_output_loss: 0.0141 - val_y_dur_output_loss: 0.0490
Epoch 5/200
36/36 [==============================] - 8s 217ms/step - loss: 0.0723 - y_note_output_loss:

In [4]:
# Save the model
model.save('./models/model_rnn_lstm.h5')

c:\Users\Niko\anaconda3\envs\ProjectVivy\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# 4. Validation
Put user input and get output

### 4.1 Input

In [5]:
import json
import joblib
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the encoding dictionary from the ph_token_to_int.json file
with open('ph_token_to_int.json', 'r') as file:
    ph_token_to_int = json.load(file)

# Input sequence
input_sequence = "AP n ei f a g e n a j i f u y u a p u AP n ei f a g e n a l e y u d ao en AP"

# Split the input sequence into individual phonemes
input_phonemes = input_sequence.split()

# Convert the phonemes to their corresponding integers using the encoding dictionary
encoded_sequence = [ph_token_to_int[phoneme] for phoneme in input_phonemes]

# Convert the sequence to a numpy array and pad it to the right length
ph_seq_encoded = np.array([encoded_sequence]) 

user_input = pad_sequences(ph_seq_encoded, maxlen=283, padding='post', truncating='post', dtype='float64')


### 4.2 Inference

In [6]:
# Prediction
y_note_pred, y_dur_pred = model.predict(user_input)
print(y_note_pred.shape, y_note_pred.shape)

1/1 [==============================] - 2s 2s/step
(1, 283, 1) (1, 283, 1)


### 4.3 Decoding

In [7]:
# Resize the result
input_length = len(encoded_sequence)
y_note_pred = y_note_pred[:, :input_length, :]
y_dur_pred = y_dur_pred[:, :input_length, :]

# print(y_note_pred.shape, y_dur_pred.shape)

# Load the saved scaler
note_scaler = joblib.load('note_scaler.pkl')

y_note_pred_flattened = y_note_pred.flatten()

# Inverse transform the predictions
y_note_pred_original = note_scaler.inverse_transform(y_note_pred_flattened.reshape(-1, 1))

# Reshape back to the original shape if required
y_note_pred_original = y_note_pred_original.reshape(y_note_pred.shape)
y_note_pred_original = np.round(y_note_pred_original).astype(int)

f0_timestep = 0.005
f0_seq_constant = 250.1

total_time = np.sum(y_dur_pred)
print(total_time)
f0_size = int(total_time / f0_timestep)
# print(f0_size)

f0_seq = [f0_seq_constant] * f0_size


14.270376


### 4.4 Make it .ds file

In [8]:
y_note_pred_flattened = y_note_pred_original[0]
y_note_pred_output = [int(i) for i in y_note_pred_flattened]

y_dur_pred_flattened = y_dur_pred[0]
y_dur_pred_output = [float(i) for i in y_dur_pred_flattened]

# Load the token-to-int mappings from the JSON files
def load_mapping(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        token_to_int = json.load(f)
    # Invert the dictionary to create an int-to-token mapping
    int_to_token = {v: k for k, v in token_to_int.items()}
    return int_to_token

# Decoding function using the mappings
def decode_predictions(prediction_integers, mapping):
    return [mapping.get(i, 'Unknown') for i in prediction_integers]

note_int_to_token = load_mapping('note_token_to_int.json')

decoded_note_seq = decode_predictions(y_note_pred_output, note_int_to_token)

# Print or return the decoded sequences
# print("Decoded Phonetic Sequence:", input_phonemes)
# print("Decoded Note Sequence:", decoded_note_seq)

ph_dur = ' '.join(map(str, y_dur_pred_output))
note_seq = ' '.join(map(str, decoded_note_seq))
f0_seq = ' '.join(map(str, f0_seq))

file = {
    'ph_seq': input_sequence,
    'ph_dur': ph_dur,
    'note_seq': note_seq,
    'f0_seq': f0_seq,
    'f0_timestep': f0_timestep
}

with open('rnn_output.ds', 'w') as json_file:
    json.dump(file, json_file, indent=4)

C:\Users\Niko\AppData\Local\Temp\ipykernel_22716\3238408921.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_note_pred_output = [int(i) for i in y_note_pred_flattened]
C:\Users\Niko\AppData\Local\Temp\ipykernel_22716\3238408921.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_dur_pred_output = [float(i) for i in y_dur_pred_flattened]
